# Image Recognition - Model Training
This notebook focuses on training a convolutional neural network (CNN) for the task of recognizing different models of "things".

In [1]:
# Import necessary libraries
import torch
import torchvision
from torchvision import transforms

## Data Loading
We load the data from local storage where images are organized into folders named after their respective classes.

In [2]:
# Define transforms and data loader
import os

transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor()
])

base_path = '../data/'
input_path = os.path.join(base_path, 'input')
test_path = os.path.join(base_path, 'test')

print("Input path: ", input_path)
print("Test path: ", test_path)

# Calculate the number of existing folder in input folder
num_folders = len([folder for folder in os.listdir(input_path)])

print("Number of folders in input folder: ", num_folders)

train_data = torchvision.datasets.ImageFolder(root=input_path, transform=transform)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)

# Repeat for test data
test_data = torchvision.datasets.ImageFolder(root=test_path, transform=transform)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)


Input path:  ../data/input
Test path:  ../data/test
Number of folders in input folder:  3


## Model Architecture
We choose a simple CNN model for this task.

In [8]:
import sys
sys.path.append('../core/')  # Add the models directory to the Python path
from simple_cnn import SimpleCNN

# Initialize the model
model = SimpleCNN(num_classes=num_folders)  # Here, 10 is the number of different classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {device}")
model.to(device)  # Move model to device


Training on cuda:0


SimpleCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=87616, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=3, bias=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

## Model Compilation
Setting up the loss function, optimizer, and metrics.

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {device}")

# Assuming model is already defined
model.to(device)

criterion = nn.CrossEntropyLoss().to(device)  # Move criterion to device
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to device

        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i % 10 == 9:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0


Training on cuda:0


## Model Export
Save the trained model.

In [16]:
# Save the model
torch.save(model.state_dict(), '../build/model.pth')